# Polars to SQL Syntax Comparison

## 0. Env

#### Imports

In [1]:
import polars as pl

In [2]:
pl.Config.set_fmt_str_lengths(90)

polars.config.Config

## 1. Create Data

In [4]:
orders_data = [
    ["2024-01-02", 50, "001"],
    ["2024-01-05", 30, "002"],
    ["2024-01-20", 44, "001"],
    ["2024-01-22", 33, "003"],
    ["2024-01-29", 25, "002"]
]
orders_schema = [
    ("order_date_utc", pl.Utf8),
    ("order_value_usd", pl.Float64),
    ("customer_id", pl.Utf8)
]
orders = pl.LazyFrame(orders_data, schema=orders_schema)

In [5]:
customers_data = [
    ["001", False, "Peter Pizza"],
    ["002", True, "Danny Dumplings"],
    ["003", True, "Barbara Burrito"]
]
customers_schema = [
    ("customer_id", pl.Utf8),
    ("is_premium_customer", pl.Boolean),
    ("name", pl.Utf8)
]
customers = pl.LazyFrame(customers_data, schema=customers_schema)

# 2. Queries

## 2.1. Query \#1: Select, Filter, and Sort

In [8]:
usd_to_eur = 1.09

print(
	orders
	.with_columns([
		pl.col("order_value_usd").mul(usd_to_eur).alias("order_value_eur")
	])
	.filter(pl.col("order_value_usd") > 30)
	.sort(by=pl.col("order_date_utc"), descending=True)
	.collect()
)

shape: (3, 4)
┌────────────────┬─────────────────┬─────────────┬─────────────────┐
│ order_date_utc ┆ order_value_usd ┆ customer_id ┆ order_value_eur │
│ ---            ┆ ---             ┆ ---         ┆ ---             │
│ str            ┆ f64             ┆ str         ┆ f64             │
╞════════════════╪═════════════════╪═════════════╪═════════════════╡
│ 2024-01-22     ┆ 33.0            ┆ 003         ┆ 35.97           │
│ 2024-01-20     ┆ 44.0            ┆ 001         ┆ 47.96           │
│ 2024-01-02     ┆ 50.0            ┆ 001         ┆ 54.5            │
└────────────────┴─────────────────┴─────────────┴─────────────────┘


## 2.2. Query \#2: Joining and Aggregating

In [9]:
print(
	orders
	.join(customers, on="customer_id", how="left")
	.group_by("name")
	.agg(pl.col("order_value_usd").sum().alias("sum_order_value_usd"))
	.collect()
)

shape: (3, 2)
┌─────────────────┬─────────────────────┐
│ name            ┆ sum_order_value_usd │
│ ---             ┆ ---                 │
│ str             ┆ f64                 │
╞═════════════════╪═════════════════════╡
│ Barbara Burrito ┆ 33.0                │
│ Peter Pizza     ┆ 94.0                │
│ Danny Dumplings ┆ 55.0                │
└─────────────────┴─────────────────────┘


## 2.3. Query \#3: CTEs and Window Functions

In [15]:
orders_w_order_rank_column = (
    orders
    .join(customers, on="customer_id", how="left")
    .with_columns([
        pl.col("order_date_utc").rank().over(pl.col("is_premium_customer")).alias("order_rank")
    ])
)

print(
    orders_w_order_rank_column
    .filter(pl.col("order_rank").le(2))
    .group_by(pl.col("is_premium_customer"))
    .agg(pl.col("order_value_usd").sum().name.prefix("sum_"))
    .collect()
)

shape: (2, 2)
┌─────────────────────┬─────────────────────┐
│ is_premium_customer ┆ sum_order_value_usd │
│ ---                 ┆ ---                 │
│ bool                ┆ f64                 │
╞═════════════════════╪═════════════════════╡
│ true                ┆ 63.0                │
│ false               ┆ 94.0                │
└─────────────────────┴─────────────────────┘
